In [10]:
import os
import json
from PIL import Image, ImageDraw, ImageFont
from surya.layout import LayoutPredictor

In [11]:
NAME = "fast_test_batch"

IMG_DIR = f"/data/BADRI/FINAL/THESIS/GRVQA/gr-doc-vqa-grounding/data/input/{NAME}/"
OUT_DIR = f"/data/BADRI/FINAL/THESIS/GRVQA/gr-doc-vqa-grounding/data/output/{NAME}/surya/"

OUT_IMG_DIR = os.path.join(OUT_DIR, "images/")
OUT_JSON_DIR = os.path.join(OUT_DIR, "json/")

if not os.path.exists(OUT_DIR):
    os.makedirs(OUT_DIR)
if not os.path.exists(OUT_IMG_DIR):
    os.makedirs(OUT_IMG_DIR)
if not os.path.exists(OUT_JSON_DIR):
    os.makedirs(OUT_JSON_DIR)


In [12]:
layout_predictor = LayoutPredictor()

Loaded layout model s3://layout/2025_02_18 on device cuda with dtype torch.float16


In [14]:
for file in os.listdir(IMG_DIR):
    IMG_PATH = os.path.join(IMG_DIR, file)


    image = Image.open(IMG_PATH)

    # layout_predictions is a list of dicts, one per image
    layout_predictions = layout_predictor([image])

    predictions = []

    for block in layout_predictions[0].bboxes:
        value = {
            'bbox': block.bbox,
            'layout': block.label,
            'confidence': block.confidence
        }

        value['bbox'] = [round(x, 2) for x in value['bbox']]
        predictions.append(value)

        # Draw bounding boxes on the image
        draw = ImageDraw.Draw(image)
        draw.rectangle(value['bbox'], outline='red', width=2)

        # draw the layout text on the image
        # font = ImageFont.truetype("arial.ttf", 16)
        draw.text((value['bbox'][0], value['bbox'][1]-10), value['layout'], fill='red')

        # Save the image with bounding boxes
        image.save(f"{OUT_IMG_DIR}/{file}")

        # save json file
        with open(f"{OUT_JSON_DIR}/{file.split('.')[0]}.json", "w") as f:
            json.dump(predictions, f, indent=4, ensure_ascii=False)


Recognizing layout: 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]
